In [30]:
import pandas as pd

In [31]:
# The path to our CSV files
schoolscsv = "schools_complete.csv"
studentscsv = "students_complete.csv"

# Read our csvfiles data into pandas
schools_df = pd.read_csv(schoolscsv)
students_df = pd.read_csv(studentscsv)

In [32]:
schools_df = schools_df.rename(columns={"name": "School Name"})

In [33]:
students_df = students_df.rename(columns={"school": "School Name"})

In [34]:
school_district = pd.merge(
    students_df, schools_df, on="School Name", suffixes=("_stu", "_sch"))

In [35]:
budget = schools_df["budget"].sum()

In [36]:
studentscount = students_df["Student ID"].count()

In [37]:
schoolscount = schools_df["School ID"].count()

In [38]:
math_mean = students_df["math_score"].mean()

In [39]:
reading_mean = students_df["reading_score"].mean()

In [40]:
district_math = students_df.loc[students_df["math_score"] >= 70,:]

In [41]:
district_reading = students_df.loc[students_df["reading_score"] >= 70,:]["reading_score"].count()

In [42]:
math_passing = round((district_math["math_score"].count()/students_df["Student ID"].count())*100,6)

In [43]:
reading_passing = round((district_reading/students_df["Student ID"].count())*100,6)

In [44]:
overall_rate = (math_passing+reading_passing)/2

In [45]:
districtsummary_df = pd.DataFrame({
    "Total Schools": schoolscount,
    "Total Students": studentscount,
    "Total Budget": budget,
    "Average Math Score": math_mean,
    "Average Reading Score": reading_mean,
    "% Passing Math": math_passing,
    "% Passing Reading": reading_passing,
    "Overall Passing Rate": overall_rate
}, index=[0])
districtsummary_df
districtsummary_df["Total Budget"] = districtsummary_df["Total Budget"].map("${:,.2f}".format)
districtsummary_df["Total Students"] = districtsummary_df["Total Students"].map("{:,}".format)

In [46]:
reorg_districtsummary = districtsummary_df[["Total Schools","Total Students","Total Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","Overall Passing Rate"]]
reorg_districtsummary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,Overall Passing Rate
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,80.393158


In [47]:
schoolgrp = school_district.groupby(["School Name"],as_index=False)

In [48]:
school_mean = pd.merge(schools_df, schoolgrp.mean(), on=['School Name'], suffixes=('_Sch','_scgrp'))
school_mean

,School ID_Sch,School Name,type,size_Sch,budget_Sch,Student ID,reading_score,math_score,School ID_scgrp,size_scgrp,budget_scgrp
0,0,Huang High School,District,2917,1910635,1458.0,81.182722,76.629414,0.0,2917.0,1910635.0
1,1,Figueroa High School,District,2949,1884411,4391.0,81.158020,76.711767,1.0,2949.0,1884411.0
2,2,Shelton High School,Charter,1761,1056600,6746.0,83.725724,83.359455,2.0,1761.0,1056600.0
3,3,Hernandez High School,District,4635,3022020,9944.0,80.934412,77.289752,3.0,4635.0,3022020.0
4,4,Griffin High School,Charter,1468,917500,12995.5,83.816757,83.351499,4.0,1468.0,917500.0
5,5,Wilson High School,Charter,2283,1319574,14871.0,83.989488,83.274201,5.0,2283.0,1319574.0
6,6,Cabrera High School,Charter,1858,1081356,16941.5,83.975780,83.061895,6.0,1858.0,1081356.0
7,7,Bailey High School,District,4976,3124928,20358.5,81.033963,77.048432,7.0,4976.0,3124928.0
8,8,Holden High School,Charter,427,248087,23060.0,83.814988,83.803279,8.0,427.0,248087.0
9,9,Pena High School,Charter,962,585858,23754.5,84.044699,83.839917,9.0,962.0,585858.0


In [49]:
del school_mean["School ID_scgrp"]

In [50]:
del school_mean["School ID_Sch"]

In [51]:
del school_mean["budget_scgrp"]
del school_mean["size_scgrp"]

In [52]:
del school_mean["Student ID"]

In [53]:
school_mean.head()

,School Name,type,size_Sch,budget_Sch,reading_score,math_score
0,Huang High School,District,2917,1910635,81.182722,76.629414
1,Figueroa High School,District,2949,1884411,81.158020,76.711767
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455
3,Hernandez High School,District,4635,3022020,80.934412,77.289752
4,Griffin High School,Charter,1468,917500,83.816757,83.351499


In [54]:
school_mean = school_mean.rename(columns={"size_Sch": "Total Students","budget_Sch": "Total Budget","reading_score": "Average Reading Score","math_score": "Average Math Score"})
school_mean

,School Name,type,Total Students,Total Budget,Average Reading Score,Average Math Score
0,Huang High School,District,2917,1910635,81.182722,76.629414
1,Figueroa High School,District,2949,1884411,81.158020,76.711767
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455
3,Hernandez High School,District,4635,3022020,80.934412,77.289752
4,Griffin High School,Charter,1468,917500,83.816757,83.351499
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895
7,Bailey High School,District,4976,3124928,81.033963,77.048432
8,Holden High School,Charter,427,248087,83.814988,83.803279
9,Pena High School,Charter,962,585858,84.044699,83.839917


In [55]:
school_mean["Per Student Budget"] = school_mean["Total Budget"]/school_mean["Total Students"]
school_mean

,School Name,type,Total Students,Total Budget,Average Reading Score,Average Math Score,Per Student Budget
0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
7,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
8,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


In [ ]:
#del school_mean["School ID_Sch"]

In [56]:
school_mean1 = pd.DataFrame(school_mean.set_index('School Name'))
school_mean

,School Name,type,Total Students,Total Budget,Average Reading Score,Average Math Score,Per Student Budget
0,Huang High School,District,2917,1910635,81.182722,76.629414,655.0
1,Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
2,Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
3,Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
4,Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
5,Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
6,Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
7,Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
8,Holden High School,Charter,427,248087,83.814988,83.803279,581.0
9,Pena High School,Charter,962,585858,84.044699,83.839917,609.0


In [57]:
school_mean1

,type,Total Students,Total Budget,Average Reading Score,Average Math Score,Per Student Budget
School Name,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,655.0
Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0
Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0
Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0
Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0
Wilson High School,Charter,2283,1319574,83.989488,83.274201,578.0
Cabrera High School,Charter,1858,1081356,83.975780,83.061895,582.0
Bailey High School,District,4976,3124928,81.033963,77.048432,628.0
Holden High School,Charter,427,248087,83.814988,83.803279,581.0


In [58]:
pass_reading_df = students_df.loc[students_df["reading_score"] >= 70,:]
pass_reading_df.head()

,Student ID,name,gender,grade,School Name,reading_score,math_score
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
4,4,Bonnie Ray,F,9th,Huang High School,97,84
5,5,Bryan Miranda,M,9th,Huang High School,94,94
6,6,Sheena Carter,F,11th,Huang High School,82,80


In [59]:
reading_passed_grp = pass_reading_df.groupby("School Name")
reading_passed_grp["Student ID"].count()

School Name
Bailey High School       4077
Cabrera High School      1803
Figueroa High School     2381
Ford High School         2172
Griffin High School      1426
Hernandez High School    3748
Holden High School        411
Huang High School        2372
Johnson High School      3867
Pena High School          923
Rodriguez High School    3208
Shelton High School      1688
Thomas High School       1591
Wilson High School       2204
Wright High School       1739
Name: Student ID, dtype: int64

In [60]:
reading_passed_grp["Student ID"].count()/school_mean1["Total Students"]

School Name
Bailey High School       0.819333
Cabrera High School      0.970398
Figueroa High School     0.807392
Ford High School         0.792990
Griffin High School      0.971390
Hernandez High School    0.808630
Holden High School       0.962529
Huang High School        0.813164
Johnson High School      0.812224
Pena High School         0.959459
Rodriguez High School    0.802201
Shelton High School      0.958546
Thomas High School       0.973089
Wilson High School       0.965396
Wright High School       0.966111
dtype: float64

In [61]:
pass_math_df = students_df.loc[students_df["math_score"] >= 70,:]
math_passed_grp = pass_math_df.groupby("School Name")
math_passed_grp["Student ID"].count()

School Name
Bailey High School       3318
Cabrera High School      1749
Figueroa High School     1946
Ford High School         1871
Griffin High School      1371
Hernandez High School    3094
Holden High School        395
Huang High School        1916
Johnson High School      3145
Pena High School          910
Rodriguez High School    2654
Shelton High School      1653
Thomas High School       1525
Wilson High School       2143
Wright High School       1680
Name: Student ID, dtype: int64

In [62]:
math_passed_grp["Student ID"].count()/school_mean1["Total Students"]

School Name
Bailey High School       0.666801
Cabrera High School      0.941335
Figueroa High School     0.659885
Ford High School         0.683096
Griffin High School      0.933924
Hernandez High School    0.667530
Holden High School       0.925059
Huang High School        0.656839
Johnson High School      0.660576
Pena High School         0.945946
Rodriguez High School    0.663666
Shelton High School      0.938671
Thomas High School       0.932722
Wilson High School       0.938677
Wright High School       0.933333
dtype: float64

In [63]:
school_mean1["% Passing Math"] = round((math_passed_grp["Student ID"].count()/school_mean1["Total Students"])*100,2)
school_mean1["% Passing Reading"] = round((reading_passed_grp["Student ID"].count()/school_mean1["Total Students"])*100,2)
school_mean1["% Overall Passing Rate"] = round((school_mean1["% Passing Math"] + school_mean1["% Passing Reading"])/2,2)
school_mean1 = school_mean1.rename(columns={"type": "School Type", "Total Budget": "Total School Budget"})
school_mean1.head()

,School Type,Total Students,Total School Budget,Average Reading Score,Average Math Score,Per Student Budget,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,81.182722,76.629414,655.0,65.68,81.32,73.50
Figueroa High School,District,2949,1884411,81.158020,76.711767,639.0,65.99,80.74,73.36
Shelton High School,Charter,1761,1056600,83.725724,83.359455,600.0,93.87,95.85,94.86
Hernandez High School,District,4635,3022020,80.934412,77.289752,652.0,66.75,80.86,73.81
Griffin High School,Charter,1468,917500,83.816757,83.351499,625.0,93.39,97.14,95.26


In [64]:
reorg_schoolsummary = school_mean1[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"]]
reorg_schoolsummary.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.50
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.36
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.87,95.85,94.86
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.81
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.26


In [65]:
reset_reorg = reorg_schoolsummary.reset_index()
reset_reorg1 = reset_reorg.sort_values("School Name")
reset_reorg1.set_index("School Name")

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.68,81.93,74.31
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.58
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.36
Ford High School,District,2739,1763916,644.0,77.102592,80.746258,68.31,79.30,73.81
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.26
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.81
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.51,96.25,94.38
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.50
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.06,81.22,73.64


In [66]:
reset_reorg2 = reset_reorg.sort_values("% Overall Passing Rate",ascending = False)
reset_reorg2.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
6,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.58
14,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.27,97.31,95.29
9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.59,95.95,95.27
4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.26
5,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.87,96.54,95.21


In [67]:
top5schools = reset_reorg2.set_index("School Name").head(5)
top5schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.58
Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.27,97.31,95.29
Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.59,95.95,95.27
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.26
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.87,96.54,95.21


In [68]:
reset_reorg3 = reset_reorg.sort_values("% Overall Passing Rate")
reset_reorg3.head(5)

,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
11,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.37,80.22,73.30
1,Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.36
0,Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.50
12,Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.06,81.22,73.64
3,Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.81


In [69]:
bottom5schools = reset_reorg3.set_index("School Name").head(5)
bottom5schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Name,,,,,,,,,
Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.37,80.22,73.30
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.36
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.50
Johnson High School,District,4761,3094650,650.0,77.072464,80.966394,66.06,81.22,73.64
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.81


In [70]:
avgscores = students_df.groupby(['School Name','grade'],as_index=False)
avgscores.mean()
avgmath_grp = avgscores["math_score"].mean()
avgmath_grp
avgmath_table =avgmath_grp.pivot('School Name','grade')
avgmath_table

math_score                                 
grade                       10th       11th       12th        9th
School Name                                                      
Bailey High School     76.996772  77.515588  76.492218  77.083676
Cabrera High School    83.154506  82.765560  83.277487  83.094697
Figueroa High School   76.539974  76.884344  77.151369  76.403037
Ford High School       77.672316  76.918058  76.179963  77.361345
Griffin High School    84.229064  83.842105  83.356164  82.044010
Hernandez High School  77.337408  77.136029  77.186567  77.438495
Holden High School     83.429825  85.000000  82.855422  83.787402
Huang High School      75.908735  76.446602  77.225641  77.027251
Johnson High School    76.691117  77.491653  76.863248  77.187857
Pena High School       83.372000  84.328125  84.121547  83.625455
Rodriguez High School  76.612500  76.395626  77.690748  76.859966
Shelton High School    82.917411  83.383495  83.778976  83.420755
Thomas High School     83.087886  83.498795  83.497041  83.590022
Wilson High School     83.724422  83.195326  83.035794  83.085578
Wright High School     84.010288  83.836782  83.644986  83.264706

In [71]:
avgmath_table[avgmath_table.columns.set_levels(['9th', '10th','11th', '12th'], level=1)]

math_score                                 
grade                        9th       10th       11th       12th
School Name                                                      
Bailey High School     77.083676  76.996772  77.515588  76.492218
Cabrera High School    83.094697  83.154506  82.765560  83.277487
Figueroa High School   76.403037  76.539974  76.884344  77.151369
Ford High School       77.361345  77.672316  76.918058  76.179963
Griffin High School    82.044010  84.229064  83.842105  83.356164
Hernandez High School  77.438495  77.337408  77.136029  77.186567
Holden High School     83.787402  83.429825  85.000000  82.855422
Huang High School      77.027251  75.908735  76.446602  77.225641
Johnson High School    77.187857  76.691117  77.491653  76.863248
Pena High School       83.625455  83.372000  84.328125  84.121547
Rodriguez High School  76.859966  76.612500  76.395626  77.690748
Shelton High School    83.420755  82.917411  83.383495  83.778976
Thomas High School     83.590022  83.087886  83.498795  83.497041
Wilson High School     83.085578  83.724422  83.195326  83.035794
Wright High School     83.264706  84.010288  83.836782  83.644986

In [72]:
avgread_grp = avgscores["reading_score"].mean()
avgread_grp
avgread_table =avgread_grp.pivot('School Name','grade')
avgread_table

reading_score                                 
grade                          10th       11th       12th        9th
School Name                                                         
Bailey High School        80.907183  80.945643  80.912451  81.303155
Cabrera High School       84.253219  83.788382  84.287958  83.676136
Figueroa High School      81.408912  80.640339  81.384863  81.198598
Ford High School          81.262712  80.403642  80.662338  80.632653
Griffin High School       83.706897  84.288089  84.013699  83.369193
Hernandez High School     80.660147  81.396140  80.857143  80.866860
Holden High School        83.324561  83.815534  84.698795  83.677165
Huang High School         81.512386  81.417476  80.305983  81.290284
Johnson High School       80.773431  80.616027  81.227564  81.260714
Pena High School          83.612000  84.335938  84.591160  83.807273
Rodriguez High School     80.629808  80.864811  80.376426  80.993127
Shelton High School       83.441964  84.373786  82.781671  84.122642
Thomas High School        84.254157  83.585542  83.831361  83.728850
Wilson High School        84.021452  83.764608  84.317673  83.939778
Wright High School        83.812757  84.156322  84.073171  83.833333

In [73]:
avgread_table[avgread_table.columns.set_levels(['9th', '10th','11th', '12th'], level=1)]

reading_score                                 
grade                           9th       10th       11th       12th
School Name                                                         
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [74]:
reorg_schoolsummary["Per Student Budget"].min()

578.0

In [75]:
reorg_schoolsummary["Per Student Budget"].max()

655.0

In [76]:
# Create bins in which to place values based upon TED Talk views
bins = [575,585,615,645,675]

# Create labels for these bins
group_labels = ["<$585", "$585-$615", "$615-$645", "$645-$675"]

perStudent_series = pd.cut(reorg_schoolsummary["Per Student Budget"], bins, labels=group_labels)
perStudent_series

School Name
Huang High School        $645-$675
Figueroa High School     $615-$645
Shelton High School      $585-$615
Hernandez High School    $645-$675
Griffin High School      $615-$645
Wilson High School           <$585
Cabrera High School          <$585
Bailey High School       $615-$645
Holden High School           <$585
Pena High School         $585-$615
Wright High School           <$585
Rodriguez High School    $615-$645
Johnson High School      $645-$675
Ford High School         $615-$645
Thomas High School       $615-$645
Name: Per Student Budget, dtype: category
Categories (4, object): [<$585 < $585-$615 < $615-$645 < $645-$675]

In [77]:
reorg_schoolsummary2 = reorg_schoolsummary
reorg_schoolsummary2["Spending ranges (Per Student)"] = perStudent_series
reorg_schoolsummary2

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending ranges (Per Student)
School Name,,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.50,$645-$675
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.36,$615-$645
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.87,95.85,94.86,$585-$615
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.81,$645-$675
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.26,$615-$645
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.87,96.54,95.21,<$585
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.58,<$585
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.68,81.93,74.31,$615-$645
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.51,96.25,94.38,<$585


In [78]:
perStudent_grp = reorg_schoolsummary2.groupby("Spending ranges (Per Student)")
#perStudent_grp.mean()
perStudent_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Spending ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460000,96.610000,95.035000
$585-$615,83.599686,83.885211,94.230000,95.900000,95.065000
$615-$645,79.079225,81.891436,75.668333,86.106667,80.888333
$645-$675,76.997210,81.027843,66.163333,81.133333,73.650000


In [79]:
reorg_schoolsummary["Total Students"].min()

427

In [80]:
reorg_schoolsummary["Total Students"].max()

4976

In [81]:
# Create bins in which to place values based upon TED Talk views
bins1 = [425,1000,2000,5000]

# Create labels for these bins
group_labels1 = ["Small(<1000)", "Medium(1000-2000)", "Large(2000-5000)"]

schoolsize_series = pd.cut(reorg_schoolsummary["Total Students"], bins1, labels=group_labels1)
schoolsize_series

School Name
Huang High School         Large(2000-5000)
Figueroa High School      Large(2000-5000)
Shelton High School      Medium(1000-2000)
Hernandez High School     Large(2000-5000)
Griffin High School      Medium(1000-2000)
Wilson High School        Large(2000-5000)
Cabrera High School      Medium(1000-2000)
Bailey High School        Large(2000-5000)
Holden High School            Small(<1000)
Pena High School              Small(<1000)
Wright High School       Medium(1000-2000)
Rodriguez High School     Large(2000-5000)
Johnson High School       Large(2000-5000)
Ford High School          Large(2000-5000)
Thomas High School       Medium(1000-2000)
Name: Total Students, dtype: category
Categories (3, object): [Small(<1000) < Medium(1000-2000) < Large(2000-5000)]

In [82]:
reorg_schoolsummary3 = reorg_schoolsummary
reorg_schoolsummary3["School Size"] = schoolsize_series
reorg_schoolsummary3

C:\Users\mywor\AppData\Local\conda\conda\envs\PythonData\lib\site-packages\ipykernel\__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Spending ranges (Per Student),School Size
School Name,,,,,,,,,,,
Huang High School,District,2917,1910635,655.0,76.629414,81.182722,65.68,81.32,73.50,$645-$675,Large(2000-5000)
Figueroa High School,District,2949,1884411,639.0,76.711767,81.158020,65.99,80.74,73.36,$615-$645,Large(2000-5000)
Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.87,95.85,94.86,$585-$615,Medium(1000-2000)
Hernandez High School,District,4635,3022020,652.0,77.289752,80.934412,66.75,80.86,73.81,$645-$675,Large(2000-5000)
Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.39,97.14,95.26,$615-$645,Medium(1000-2000)
Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.87,96.54,95.21,<$585,Large(2000-5000)
Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.13,97.04,95.58,<$585,Medium(1000-2000)
Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.68,81.93,74.31,$615-$645,Large(2000-5000)
Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.51,96.25,94.38,<$585,Small(<1000)


In [83]:
schoolsize_grp = reorg_schoolsummary3.groupby("School Size")
#perStudent_grp.mean()
schoolsize_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Size,,,,,
Small(<1000),83.821598,83.929843,93.55000,96.10000,94.8250
Medium(1000-2000),83.374684,83.864438,93.59800,96.79000,95.1920
Large(2000-5000),77.746417,81.344493,69.96375,82.76625,76.3675


In [84]:
schooltype_grp = reorg_schoolsummary.groupby("School Type")
schooltype_grp["Average Math Score","Average Reading Score","% Passing Math","% Passing Reading","% Overall Passing Rate"].mean()

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,93.620000,96.586250,95.102500
District,76.956733,80.966636,66.548571,80.798571,73.675714
